<a href="https://colab.research.google.com/github/Takumi173/DifyApps/blob/main/Dataset_JSON_Reviewer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備

## 処理用にデータを結合

In [1]:
# データのコピー
!git clone https://github.com/cdisc-org/sdtm-adam-pilot-project.git

Cloning into 'sdtm-adam-pilot-project'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (150/150), done.
remote: Total 224 (delta 64), reused 220 (delta 61), pack-reused 0 (from 0)
Receiving objects: 100% (224/224), 24.51 MiB | 4.92 MiB/s, done.
Resolving deltas: 100% (64/64), done.
Updating files: 100% (87/87), done.


In [2]:
# 使用するjsonデータとdefine.xmlを新規ディレクトリにコピーする

import os
import shutil
import json

source_dir = "sdtm-adam-pilot-project/updated-pilot-submission-package/900172/m5/datasets/cdiscpilot01/tabulations/sdtm"
json_dir   = "json_files"
define_dir = "define_xml"

if not os.path.exists(json_dir):
    os.makedirs(json_dir)

if not os.path.exists(define_dir):
    os.makedirs(define_dir)

for root, _, files in os.walk(source_dir):
  for file in files:
    if file.endswith(".json"):
      source_path = os.path.join(root, file)
      target_path = os.path.join(json_dir, file)
      shutil.copy(source_path, target_path)
    if file.endswith("define.xml"):
      source_path = os.path.join(root, file)
      target_path = os.path.join(define_dir, file)
      shutil.copy(source_path, target_path)

In [3]:
# jsonファイルをリスト形式に結合したファイル（dataset_list.json）を作成

dataset_list = []
for filename in os.listdir(json_dir):
  if filename.endswith(".json"):
    with open(os.path.join(json_dir, filename), "r") as f:
      try:
        json_data = json.load(f)
        dataset_list.append(json_data)
      except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {filename}: {e}")

with open("dataset_list.json", "w") as f:
  json.dump(dataset_list, f)


## 症例フィルタリング関数の定義

In [4]:
def filter_data(data, target_usubjids):
    """
    複数のドメインデータを含むリストから、指定されたUSUBJIDのrowsのみを抽出して新しいJSONファイルに保存する。
    入力データがリストでない場合はエラーメッセージを出力する。
    データ構造は、"columns" 内の "name" が "USUBJID" の列を持つことを前提とする。

    Args:
        data (list): ドメインを結合させたのリスト。リストでない場合はエラーとなる。
        output_file (str): 出力するJSONファイル名。
        target_usubjids (list): 残したいUSUBJIDのリスト。
    """
    if not isinstance(data, list):
        print("エラー：入力データはJSONオブジェクトのリストである必要があります。")
        return

    filtered_data_list = []
    for item in data:
        usubjid_index = -1
        if 'columns' in item:
            for i, col in enumerate(item['columns']):
                if 'name' in col and col['name'] == 'USUBJID':
                    usubjid_index = i
                    break

        if usubjid_index == -1:
            print(f"警告：データセット '{item.get('fileOID', '不明')}' に 'name' が 'USUBJID' の列が見つかりません。スキップします。")
            filtered_data_list.append(item)
            continue

        if 'rows' in item:
            filtered_rows = [
                row for row in item['rows'] if len(row) > usubjid_index and row[usubjid_index] in target_usubjids
            ]
            new_data = item.copy()
            new_data['rows'] = filtered_rows
            new_data['records'] = len(filtered_rows)
            filtered_data_list.append(new_data)
        else:
            print(f"警告：データセット '{item.get('fileOID', '不明')}' に 'rows' が見つかりません。スキップします。")
            filtered_data_list.append(item)

    return filtered_data_list



# 実行テスト
# with open('dataset_list.json', 'r') as f:
#   data = json.load(f)
#
# target_ids = ['01-701-1211']
# output_filename = 'filtered_list.json'
#
# filtered_data_list = filter_data(data, target_ids)
#
# with open(output_filename, 'w') as f:
#   json.dump(filtered_data_list, f)
#
# print(f"処理完了：'{output_filename}' に USUBJID が {target_ids} のデータを出力しました。")

## データ書き換え関数の定義

In [5]:
def data_update(data, target_domain, target_usubjid, target_seq, target_variable, new_value):
    """
    指定されたUSUBJIDを持つレコードの指定された変数を書き換えます。
    {target_domain}SEQが存在する場合はそれもキーとして使用します。
    元のデータは変更せず、新しいデータ構造を返します。

    Args:
        data (list): データ全体のリスト。指定されたtarget_domainのデータセットを含むことを想定します。
        target_domain (str): 対象のドメイン名（例: "CM"）。
        target_usubjid (str): 書き換えたいレコードのUSUBJID。
        target_seq (int): 書き換えたいレコードの{target_domain}SEQの値（存在しない場合は無視されます）。
        target_variable (str): 書き換えたい変数の名前（例: "CMTRT"）。
        new_value (any): 新しい変数の値。

    Returns:
        list: 指定された変数が更新された新しいデータ全体のリスト。
              該当するレコードが見つからなかった場合、元のデータのコピーを返します。
    """
    updated_data = []
    seqname = target_domain + 'SEQ'

    for dataset in data:
        updated_dataset = dataset.copy()
        if updated_dataset.get("itemGroupOID") == target_domain:
            updated_rows = []
            found = False
            usubjid_index = -1
            seq_index = -1
            variable_index = -1
            has_seq = False

            for i, col in enumerate(updated_dataset["columns"]):
                if col["name"] == "USUBJID":
                    usubjid_index = i
                elif col["name"] == seqname:
                    seq_index = i
                    has_seq = True
                elif col["name"] == target_variable:
                    variable_index = i

            if usubjid_index != -1 and variable_index != -1:
                for row in dataset["rows"]:
                    updated_row = list(row)  # 行をコピーして変更
                    usubjid_match = updated_row[usubjid_index] == target_usubjid
                    seq_match = True
                    if has_seq and seq_index != -1:
                        seq_match = (len(updated_row) > seq_index and updated_row[seq_index] == target_seq)
                    elif has_seq:
                        print(f"警告: '{target_domain}' データセットに '{seqname}' 列が見つかりましたが、インデックスが無効です。USUBJIDのみをキーとして使用します。")

                    if usubjid_match and seq_match:
                        updated_row[variable_index] = new_value
                        if has_seq and seq_index != -1:
                            print(f"USUBJID '{target_usubjid}'、'{seqname}' '{target_seq}' の '{target_variable}' を '{new_value}' に更新しました。")
                        else:
                            print(f"USUBJID '{target_usubjid}' の '{target_variable}' を '{new_value}' に更新しました。")
                        found = True
                    updated_rows.append(updated_row)
                updated_dataset["rows"] = updated_rows
            elif updated_dataset.get("itemGroupOID") == target_domain:
                print(f"'{target_domain}' データセットに 'USUBJID' または '{target_variable}' 列が見つかりませんでした。")

            updated_data.append(updated_dataset)
            if not found and updated_dataset.get("itemGroupOID") == target_domain:
                if has_seq and seq_index != -1:
                    print(f"USUBJID '{target_usubjid}'、'{seqname}' '{target_seq}' に該当するレコードが見つかりませんでした。")
                else:
                    print(f"USUBJID '{target_usubjid}' に該当するレコードが見つかりませんでした。")
        else:
            updated_data.append(updated_dataset)

    if not any(d.get("itemGroupOID") == target_domain for d in data):
        print(f"{target_domain} データセットが見つかりませんでした。")

    return updated_data

# 書き換えテスト
# updated_data = data_update(filtered_data_list, "DM", "01-701-1211", 0, "AGE", 49)
# updated_data = data_update(updated_data, "CM", "01-701-1211", 3, "CMTRT", "New Drug 123456789")
# updated_data = data_update(updated_data, "CM", "01-701-1211", 0, "CMDOSE", 123)

## データ比較関数の定義

In [6]:
from typing import List, Dict, Any

def compare_data(old_data: List[Dict[str, Any]], new_data: List[Dict[str, Any]]) -> None:
    """
    2つのデータリストの更新差分を人間が読みやすい形式で出力します。

    Args:
        old_data: 旧データリスト。
        new_data: 新データリスト。
    """

    def create_row_dict(item_group: Dict[str, Any], row: List[Any]) -> Dict[str, Any]:
        """rowデータをキー付きの辞書に変換する"""
        row_dict = {}
        for i, column in enumerate(item_group['columns']):
            row_dict[column['name']] = row[i]
        return row_dict

    def get_key_values(item_group_oid: str, row_dict: Dict[str, Any]) -> Dict[str, Any]:
        """データのキーとなる値を抽出する"""
        key_values = {'USUBJID': row_dict.get('USUBJID')}
        seq_key = f"{item_group_oid}SEQ"
        if seq_key in row_dict:
            key_values[seq_key] = row_dict[seq_key]
        return key_values

    def format_key(key_values: Dict[str, Any]) -> str:
        """キー値を人間が読みやすい文字列に整形する"""
        parts = []
        for key, value in key_values.items():
            if value is not None:
                parts.append(f"{key} = {value}")
        return ", ".join(parts)

    old_data_by_group = {item['itemGroupOID']: item for item in old_data}
    new_data_by_group = {item['itemGroupOID']: item for item in new_data}

    all_group_oids = set(old_data_by_group.keys()) | set(new_data_by_group.keys())

    for group_oid in sorted(list(all_group_oids)):
        print(f"--- ItemGroupOID: {group_oid} ---")
        old_group = old_data_by_group.get(group_oid)
        new_group = new_data_by_group.get(group_oid)

        old_rows_by_key = {}
        if old_group and 'rows' in old_group:
            for row in old_group['rows']:
                row_dict = create_row_dict(old_group, row)
                if 'USUBJID' in row_dict and row_dict['USUBJID'] is not None:
                    key_values = get_key_values(group_oid, row_dict)
                    old_rows_by_key[format_key(key_values)] = row_dict

        new_rows_by_key = {}
        if new_group and 'rows' in new_group:
            for row in new_group['rows']:
                row_dict = create_row_dict(new_group, row)
                if 'USUBJID' in row_dict and row_dict['USUBJID'] is not None:
                    key_values = get_key_values(group_oid, row_dict)
                    new_rows_by_key[format_key(key_values)] = row_dict

        old_keys = set(old_rows_by_key.keys())
        new_keys = set(new_rows_by_key.keys())

        # 追加されたデータ
        added_keys = new_keys - old_keys
        for key in sorted(list(added_keys)):
            print(f"{key}:")
            print("  Added")
            for item_key, old_value in sorted(new_rows_by_key[key].items()):
                print(f"    {item_key}: {old_value}")
            print()

        # 削除されたデータ
        removed_keys = old_keys - new_keys
        for key in sorted(list(removed_keys)):
            print(f"{key}:")
            print("  Deleted")
            for item_key, old_value in sorted(old_rows_by_key[key].items()):
                print(f"    {item_key}: {old_value}")
            print()

        # 更新されたデータ
        common_keys = old_keys & new_keys
        for key in sorted(list(common_keys)):
            if old_rows_by_key[key] != new_rows_by_key[key]:
                print(f"{key}:")
                print("  Updated:")
                old_row = old_rows_by_key[key]
                new_row = new_rows_by_key[key]
                for item_key in sorted(list(set(old_row.keys()) | set(new_row.keys()))):
                    old_value = old_row.get(item_key)
                    new_value = new_row.get(item_key)
                    if old_value != new_value:
                        print(f"    {item_key}: {old_value!r} -> {new_value!r}")
                print()


# 比較テスト
# compare_data(filtered_data_list, updated_data)

In [7]:
import json

usubjids = set()
for dataset in dataset_list:
    if 'columns' in dataset:
        for i, col in enumerate(dataset['columns']):
            if 'name' in col and col['name'] == 'USUBJID':
                if 'rows' in dataset:
                    for row in dataset['rows']:
                        if len(row) > i:
                            usubjids.add(row[i])

print(list(usubjids))


['01-710-1077', '01-703-1119', '01-703-1439', '01-702-1082', '01-710-1045', '01-717-1446', '01-701-1275', '01-708-1297', '01-718-1355', '01-716-1311', '01-718-1328', '01-701-1057', '01-708-1216', '01-701-1387', '01-716-1094', '01-710-1376', '01-703-1076', '01-711-1012', '01-701-1111', '01-709-1238', '01-701-1203', '01-718-1172', '01-707-1276', '01-711-1163', '01-708-1171', '01-715-1333', '01-708-1296', '01-709-1001', '01-710-1249', '01-718-1371', '01-710-1129', '01-709-1217', '01-701-1392', '01-710-1300', '01-718-1427', '01-704-1332', '01-701-1363', '01-711-1433', '01-704-1093', '01-704-1325', '01-710-1002', '01-716-1108', '01-717-1109', '01-704-1388', '01-705-1431', '01-705-1349', '01-701-1028', '01-708-1178', '01-716-1244', '01-703-1403', '01-708-1378', '01-704-1233', '01-704-1260', '01-713-1043', '01-709-1339', '01-703-1210', '01-708-1428', '01-709-1088', '01-716-1026', '01-704-1074', '01-703-1258', '01-714-1195', '01-710-1137', '01-714-1288', '01-716-1071', '01-708-1032', '01-704-1

# データの書き換え

In [8]:
Target_data = [
["DM", "01-703-1096",   0, "AGE", 49],
["LB", "01-703-1042",   3, "LBORRES", "135"],
["LB", "01-703-1042",   4, "LBORRES", "145"],
["LB", "01-703-1086",  37, "LBORRES", "1"],
["LB", "01-703-1086",  72, "LBORRES", "1.2"],
["LB", "01-703-1086", 102, "LBORRES", "1.1"],
["LB", "01-703-1086", 132, "LBORRES", "1"],
["LB", "01-703-1086", 162, "LBORRES", "1.3"],
["LB", "01-703-1086", 197, "LBORRES", "0.9"],
["LB", "01-703-1086", 232, "LBORRES", "0.8"],
["LB", "01-703-1042",   3, "LBSTRESC", "135"],
["LB", "01-703-1042",   4, "LBSTRESC", "145"],
["LB", "01-703-1086",  37, "LBSTRESC", "1"],
["LB", "01-703-1086",  72, "LBSTRESC", "1.2"],
["LB", "01-703-1086", 102, "LBSTRESC", "1.1"],
["LB", "01-703-1086", 132, "LBSTRESC", "1"],
["LB", "01-703-1086", 162, "LBSTRESC", "1.3"],
["LB", "01-703-1086", 197, "LBSTRESC", "0.9"],
["LB", "01-703-1086", 232, "LBSTRESC", "0.8"],
["LB", "01-703-1042",   3, "LBSTRESN", 135],
["LB", "01-703-1042",   4, "LBSTRESN", 145],
["LB", "01-703-1086",  37, "LBSTRESN", 1],
["LB", "01-703-1086",  72, "LBSTRESN", 1.2],
["LB", "01-703-1086", 102, "LBSTRESN", 1.1],
["LB", "01-703-1086", 132, "LBSTRESN", 1],
["LB", "01-703-1086", 162, "LBSTRESN", 1.3],
["LB", "01-703-1086", 197, "LBSTRESN", 0.9],
["LB", "01-703-1086", 232, "LBSTRESN", 0.8],
["LB", "01-703-1042",   3, "LBNRIND", "HIGH"],
["LB", "01-703-1042",   4, "LBNRIND", "HIGH"],
["LB", "01-703-1086",  37, "LBNRIND", "LOW"],
["LB", "01-703-1086",  72, "LBNRIND", "LOW"],
["LB", "01-703-1086", 102, "LBNRIND", "LOW"],
["LB", "01-703-1086", 132, "LBNRIND", "LOW"],
["LB", "01-703-1086", 162, "LBNRIND", "LOW"],
["LB", "01-703-1086", 197, "LBNRIND", "LOW"],
["LB", "01-703-1086", 232, "LBNRIND", "LOW"],
["MH", "01-701-1097",   1, "MHTERM", "Loss of consciousness (Passed out)"],
["MH", "01-701-1097",   1, "MHSTDTC", "2023-01-01"],
["MH", "01-701-1111",   1, "MHTERM", "HEARING LOSS"],
["MH", "01-701-1180",   1, "MHTERM", "DEPRESSION (ANXIETY)"],
["MH", "01-702-1082",   1, "MHTERM", "Premenstrual pain"],
["MH", "01-703-1076",   1, "MHTERM", "Atrioventricular block (scheduled cardiac pacemaker insertion)"],
["MH", "01-703-1279",   1, "MHTERM", "schizophreniform disorders"],
["MH", "01-703-1299",   1, "MHTERM", "Cyclothymic disorder"],
["VS", "01-701-1047",  17, "VSORRES", "121"],
["VS", "01-701-1047",  18, "VSORRES", "124"],
["VS", "01-701-1047",  66, "VSORRES", "185"],
["VS", "01-701-1047",  67, "VSORRES", "183"],
["VS", "01-701-1383",  37, "VSORRES", "98"],
["VS", "01-701-1383", 122, "VSORRES", "160"],
["VS", "01-701-1387",   1, "VSORRES", "146"],
["VS", "01-701-1387",  32, "VSORRES", "72"],
["VS", "01-701-1047",  17, "VSSTRESC", "121"],
["VS", "01-701-1047",  18, "VSSTRESC", "124"],
["VS", "01-701-1047",  66, "VSSTRESC", "185"],
["VS", "01-701-1047",  67, "VSSTRESC", "183"],
["VS", "01-701-1383",  37, "VSSTRESC", "98"],
["VS", "01-701-1383", 122, "VSSTRESC", "160"],
["VS", "01-701-1387",   1, "VSSTRESC", "146"],
["VS", "01-701-1387",  32, "VSSTRESC", "72"],
["VS", "01-701-1047",  17, "VSSTRESN", 121],
["VS", "01-701-1047",  18, "VSSTRESN", 124],
["VS", "01-701-1047",  66, "VSSTRESN", 185],
["VS", "01-701-1047",  67, "VSSTRESN", 183],
["VS", "01-701-1383",  37, "VSSTRESN", 98],
["VS", "01-701-1383", 122, "VSSTRESN", 160],
["VS", "01-701-1387",   1, "VSSTRESN", 146],
["VS", "01-701-1387",  32, "VSSTRESN", 72],
["EX", "01-701-1148",   2, "EXDOSE", 82],
["EX", "01-701-1148",   3, "EXDOSE", 216],
["EX", "01-703-1258",   2, "EXDOSE", 27],
["CM", "01-701-1146",  29, "CMTRT", "PAROXETINE"],
["QS", "01-701-1023",1010, "QSORRES", "PRESENT"],
["QS", "01-701-1023",1012, "QSORRES", "PRESENT"],
["QS", "01-701-1111",5004, "QSORRES", "4"],
["QS", "01-701-1111",5019, "QSORRES", "4"],
["QS", "01-701-1111",5012, "QSORRES", "4"],
["QS", "01-701-1111",5027, "QSORRES", "4"],
["QS", "01-701-1118",6002, "QSORRES", "MARKED IMPROVEMENT"],
["QS", "01-701-1118",6003, "QSORRES", "MARKED WORSENING"],
["QS", "01-701-1181",4018, "QSORRES", "Y"],
["QS", "01-701-1181",4058, "QSORRES", "Y"],
["QS", "01-701-1181",4019, "QSORRES", "Y"],
["QS", "01-701-1181",4059, "QSORRES", "Y"],
["QS", "01-701-1181",4020, "QSORRES", "Y"],
["QS", "01-701-1023",1010, "QSSTRESC", "2"],
["QS", "01-701-1023",1012, "QSSTRESC", "2"],
["QS", "01-701-1111",5004, "QSSTRESC", "4"],
["QS", "01-701-1111",5019, "QSSTRESC", "4"],
["QS", "01-701-1111",5012, "QSSTRESC", "4"],
["QS", "01-701-1111",5027, "QSSTRESC", "4"],
["QS", "01-701-1118",6002, "QSSTRESC", "1"],
["QS", "01-701-1118",6003, "QSSTRESC", "7"],
["QS", "01-701-1181",4018, "QSSTRESC", "1"],
["QS", "01-701-1181",4058, "QSSTRESC", "1"],
["QS", "01-701-1181",4019, "QSSTRESC", "1"],
["QS", "01-701-1181",4059, "QSSTRESC", "1"],
["QS", "01-701-1181",4020, "QSSTRESC", "1"],
["QS", "01-701-1023",1010, "QSSTRESN", 2],
["QS", "01-701-1023",1012, "QSSTRESN", 2],
["QS", "01-701-1111",5004, "QSSTRESN", 4],
["QS", "01-701-1111",5019, "QSSTRESN", 4],
["QS", "01-701-1111",5012, "QSSTRESN", 4],
["QS", "01-701-1111",5027, "QSSTRESN", 4],
["QS", "01-701-1118",6002, "QSSTRESN", 1],
["QS", "01-701-1118",6003, "QSSTRESN", 7],
["QS", "01-701-1181",4018, "QSSTRESN", 1],
["QS", "01-701-1181",4058, "QSSTRESN", 1],
["QS", "01-701-1181",4019, "QSSTRESN", 1],
["QS", "01-701-1181",4059, "QSSTRESN", 1],
["QS", "01-701-1181",4020, "QSSTRESN", 1],
["QS", "01-701-1118",6001, "QSDTC", "2014-07-08"],
["QS", "01-701-1118",6001, "QSDY", 119],
["AE", "01-701-1015",   3, "AESER", "Y"],
["AE", "01-701-1015",   3, "AESHOSP", "Y"],
["AE", "01-701-1015",   3, "AESTDTC", "2014-01-11"],
["AE", "01-701-1015",   3, "AEENDTC", "2014-01-09"],
["AE", "01-701-1015",   3, "AESTDY", 10],
["AE", "01-701-1015",   3, "AEENDY", 8],
["AE", "01-701-1028",   1, "AETERM", "PARKINSON'S DISEASE"],
["AE", "01-701-1028",   1, "AESTDTC", "2013-07-01"],
["AE", "01-701-1028",   1, "AESTDY", -17],
["AE", "01-701-1034",   2, "AETERM", "MALIGNANT HYPERTENSION"],
["AE", "01-701-1047",   4, "AETERM", "HYPERTENSION"],
["AE", "01-701-1363",   1, "AESTDTC", "2013-06-15"],
["AE", "01-701-1363",   1, "AEENDTC", "2013-06-14"],
["AE", "01-701-1363",   1, "AESTDY", 17],
["AE", "01-701-1363",   1, "AEENDY", 16],
["AE", "01-701-1047",   3, "AEENDTC", "2013-03-05"],
["AE", "01-701-1047",   3, "AEENDY", 22],
["AE", "01-701-1383",  12, "AETERM", "BLOOD PRESSURE INCREASED"],
["AE", "01-701-1153",   2, "AEACN", "DRUG WITHDRAWN"],
["AE", "01-701-1180",   6, "AETERM", "SUDDEN DEATH"],
["AE", "01-703-1258",   2, "AESEV", "SEVERE"],
["AE", "01-703-1258",   2, "AESTDTC", "2012-08-01"],
["AE", "01-703-1258",   2, "AEENDTC", "2012-10-01"],
["AE", "01-703-1258",   2, "AESTDY", 13],
["AE", "01-703-1258",   2, "AEENDY", 74],
["AE", "01-703-1258",   5, "AESEV", "MODERATE"],
["AE", "01-703-1258",   5, "AESER", "Y"],
["AE", "01-703-1258",   5, "AEOUT", "RECOVERED/RESOLVED"],
["AE", "01-703-1258",   5, "AESLIFE", "Y"],
["AE", "01-703-1258",   5, "AESTDTC", "2012-10-02"],
["AE", "01-703-1258",   5, "AEENDTC", "2012-12-31"],
["AE", "01-703-1258",   2, "AESTDY", 75],
["AE", "01-703-1258",   2, "AEENDY", 165],
["AE", "01-703-1335",   1, "AETERM", "MULTIPLE SCLEROSIS RELAPSE"],
["AE", "01-703-1335",   1, "AESTDTC", "2014-04-01"],
["AE", "01-703-1335",   1, "AEENDTC", "2014-05-01"],
["AE", "01-703-1335",   1, "AESTDY", 15],
["AE", "01-703-1335",   1, "AEENDY", 46],
["AE", "01-703-1403",   2, "AETERM", "MYASTHENIA GRAVIS AGGRAVATED"],
["AE", "01-704-1008",   1, "AETERM", "TREMOR IN HANDS, LEGS"],
["AE", "01-704-1008",   1, "AEREL", "NONE"],
["AE", "01-704-1008",   1, "AESTDTC", "2012-06-01"],
["AE", "01-704-1008",   1, "AESTDY", -225],
["AE", "01-704-1008",   3, "AETERM", "MUSCLE STIFFNESS"],
["AE", "01-704-1008",   3, "AESTDTC", "2012-06-01"],
["AE", "01-704-1008",   3, "AESTDY", -225],
["AE", "01-704-1008",   2, "AETERM", "SLOWNESS of MOVEMENT"],
["AE", "01-704-1008",   2, "AESTDTC", "2012-06-01"],
["AE", "01-704-1008",   2, "AESTDY", -225],
["AE", "01-704-1009",   6, "AETERM", "CHRONIC KIDNEY DISEASE"],
["AE", "01-704-1009",   6, "AESER", "Y"],
["AE", "01-704-1009",   6, "AESLIFE", "Y"],
["AE", "01-704-1010",   1, "AETERM", "DIABETES MELLITUS"],
["AE", "01-704-1010",   1, "AESER", "Y"],
["AE", "01-704-1010",   1, "AESLIFE", "Y"],
["AE", "01-704-1017",   4, "AETERM", "LATE EFFECTS OF CEREBRAL INFRACTION"],
["AE", "01-704-1017",   4, "AESEV", "SEVERE",],
["AE", "01-704-1017",   4, "AESTDTC", "2013-10-19"],
["AE", "01-704-1017",   4, "AEENDTC", "2013-11-18"],
["AE", "01-704-1017",   4, "AESTDY", 14],
["AE", "01-704-1017",   4, "AEENDY", 44],
["AE", "01-704-1017",   3, "AETERM", "BRAIN DEATH"],
["AE", "01-704-1017",   3, "AESEV", "SEVERE",],
["AE", "01-704-1017",   3, "AESTDTC", "2013-11-18"],
["AE", "01-704-1017",   3, "AEENDTC", "2013-11-18"],
["AE", "01-704-1017",   3, "AESTDY", 44],
["AE", "01-704-1017",   3, "AEENDY", 44],
["AE", "01-704-1017",   1, "AEOUT", "RECOVERED/RESOLVED"],
["AE", "01-704-1017",   1, "AESTDTC", "2013-10-19"],
["AE", "01-704-1017",   1, "AEENDTC", "2013-11-19"],
["AE", "01-704-1017",   1, "AESTDY", 14],
["AE", "01-704-1017",   1, "AEENDY", 45],
["AE", "01-704-1017",   1, "AEACN", "DRUG WITHDRAWN"]
]

dataset_list_updated = dataset_list

for l in Target_data:
  #print(l)
  dataset_list_updated = data_update(dataset_list_updated, l[0], l[1], l[2], l[3], l[4])

USUBJID '01-703-1096' の 'AGE' を '49' に更新しました。
USUBJID '01-703-1042'、'LBSEQ' '3' の 'LBORRES' を '135' に更新しました。
USUBJID '01-703-1042'、'LBSEQ' '4' の 'LBORRES' を '145' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '37' の 'LBORRES' を '1' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '72' の 'LBORRES' を '1.2' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '102' の 'LBORRES' を '1.1' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '132' の 'LBORRES' を '1' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '162' の 'LBORRES' を '1.3' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '197' の 'LBORRES' を '0.9' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '232' の 'LBORRES' を '0.8' に更新しました。
USUBJID '01-703-1042'、'LBSEQ' '3' の 'LBSTRESC' を '135' に更新しました。
USUBJID '01-703-1042'、'LBSEQ' '4' の 'LBSTRESC' を '145' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '37' の 'LBSTRESC' を '1' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '72' の 'LBSTRESC' を '1.2' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '102' の 'LBSTRESC' を '1.1' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '132' の 'LBSTRESC' を '1' に更

In [9]:
# 更新データ確認
compare_data(dataset_list, dataset_list_updated)

with open("dataset_list_updated.json", "w") as f:
  json.dump(dataset_list_updated, f)

--- ItemGroupOID: AE ---
USUBJID = 01-701-1015, AESEQ = 3:
  Updated:
    AEENDTC: '2014-01-11' -> '2014-01-09'
    AEENDY: 10 -> 8
    AESER: 'N' -> 'Y'
    AESHOSP: 'N' -> 'Y'
    AESTDTC: '2014-01-09' -> '2014-01-11'
    AESTDY: 8 -> 10

USUBJID = 01-701-1028, AESEQ = 1:
  Updated:
    AESTDTC: '2013-07-21' -> '2013-07-01'
    AESTDY: 3 -> -17
    AETERM: 'APPLICATION SITE ERYTHEMA' -> "PARKINSON'S DISEASE"

USUBJID = 01-701-1034, AESEQ = 2:
  Updated:
    AETERM: 'FATIGUE' -> 'MALIGNANT HYPERTENSION'

USUBJID = 01-701-1047, AESEQ = 3:
  Updated:
    AEENDTC: '' -> '2013-03-05'
    AEENDY: None -> 22

USUBJID = 01-701-1047, AESEQ = 4:
  Updated:
    AETERM: 'BUNDLE BRANCH BLOCK LEFT' -> 'HYPERTENSION'

USUBJID = 01-701-1153, AESEQ = 2:
  Updated:
    AEACN: '' -> 'DRUG WITHDRAWN'

USUBJID = 01-701-1180, AESEQ = 6:
  Updated:
    AETERM: 'MICTURITION URGENCY' -> 'SUDDEN DEATH'

USUBJID = 01-701-1363, AESEQ = 1:
  Updated:
    AEENDTC: '2013-06-15' -> '2013-06-14'
    AEENDY: 17 -> 16

# LLMへの送信

In [10]:
!pip install sseclient-py
import requests
import sseclient
from IPython.display import display, Markdown

from google.colab import userdata
api_key = userdata.get('Dify_DatasetJSON')
user_id = 'JPMA_Sample'

## 関数定義

In [11]:
import json
import time
import requests
import sseclient

# 定数
DIFY_API_URL = 'https://api.dify.ai/v1/workflows/run'
CONTENT_TYPE_JSON = 'application/json'

def call_dify_api(api_key: str, payload: dict, stream: bool = False) -> requests.Response:
    """Dify APIを呼び出す共通関数"""
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': CONTENT_TYPE_JSON
    }
    try:
        response = requests.post(DIFY_API_URL, headers=headers, json=payload, stream=stream)
        response.raise_for_status()  # HTTPエラーが発生した場合に例外を発生させる
        return response
    except requests.exceptions.RequestException as e:
        print(f"API呼び出しエラー: {e}")
        raise

def run_dify_workflow(api_key: str, workflow_inputs: dict, user_id: str, streaming: bool = False) -> dict | sseclient.Event:
    """Difyワークフローを実行する

    Args:
        api_key: Dify APIキー
        workflow_inputs: ワークフローへの入力
        user_id: ユーザーID
        streaming: ストリーミングモードで実行するかどうか (Falseの場合はブロッキングモード)

    Returns:
        ストリーミングモードの場合はsseclient.Eventのイテレータ、
        ブロッキングモードの場合はAPIのレスポンスのJSONを辞書型で返す
    """
    payload = {
        'inputs': workflow_inputs,
        'response_mode': 'streaming' if streaming else 'blocking',
        'user': user_id
    }
    response = call_dify_api(api_key, payload, stream=streaming)
    if streaming:
        client = sseclient.SSEClient(response)
        return client.events()
    else:
        return response.json()

def safe_print_event_data(event: sseclient.Event):
    """
    与えられたSSEイベントデータから、存在する場合に特定の値を出力します。
    キーが存在しない場合は何も出力しません。Statusが存在する場合にのみTitleと結合させて表示します。

    Args:
        event: イベントデータを含むSSEイベントオブジェクト。event.data属性がJSON文字列であることを想定。
    """
    try:
        data = json.loads(event.data)

        if 'event' in data:
            print(f"Event: {data['event']}")

        if 'data' in data:
            title = data['data'].get('title')
            status = data['data'].get('status')
            if title is not None and status is not None:
                print(f"Node: {title} ({status})")
            elif title is not None:
                print(f"Node: {title}") # Statusが存在しない場合はTitleのみ表示

            if 'error' in data['data']:
                print(f"Error: {data['data']['error']}")
            if 'elapsed_time' in data['data']:
                print(f"Elapsed time: {data['data'].get('elapsed_time')}")
            if 'total_tokens' in data['data']:
                print(f"Total tokens: {data['data'].get('total_tokens')}")

    except json.JSONDecodeError as e:
        print(f"Error decoding JSON event data: {e}")
    except AttributeError as e:
        print(f"Error accessing event data attribute: {e}")

def run_workflow_with_retry(api_key: str, workflow_inputs: dict, user_id: str, max_retries: int = 3, retry_delay: int = 30):
    """ワークフローを実行し、エラー発生時にリトライを行う (ストリーミングモード専用)"""
    for retry in range(max_retries + 1):
        print(f"--- 試行回数: {retry + 1} ---")
        success = True
        try:
            for event in run_dify_workflow(api_key, workflow_inputs, user_id, streaming=True):
                safe_print_event_data(event)
                try:
                    event_data = json.loads(event.data)
                    if event_data.get('data', {}).get('error') is not None:
                        print(f"エラーが検出されました: {event_data['data']['error']}")
                        success = False
                        break
                except json.JSONDecodeError:
                    print("JSONデコードエラーが発生しました。")
                    success = False
                    break
                print('------')

            if success:
                print("ワークフローが正常に完了しました。")
                return json.loads(event.data)
            elif retry < max_retries:
                print(f"エラーが発生したため、{retry_delay}秒後に再試行します...")
                time.sleep(retry_delay)
            else:
                print("最大再試行回数に達しました。ワークフローは失敗しました。")
                return False

        except requests.exceptions.RequestException as e:
            print(f"APIリクエスト中にエラーが発生しました: {e}")
            success = False
            if retry < max_retries:
                print(f"{retry_delay}秒後に再試行します...")
                time.sleep(retry_delay)
            else:
                print("最大再試行回数に達しました。ワークフローは失敗しました。")
                return False

## プロンプトの作成

In [12]:
with open('define_xml/define.xml', 'r') as f:
  define_xml = f.read()


SysPrompt = '''
あなたは、臨床試験データのレビューを支援するAIアシスタントです。以下の前提知識を理解した上で、ユーザーからの指示（ユーザープロンプト）に従って、臨床試験データのレビューを支援してください。各タスクでは、ユーザープロンプトで指定された役割になりきって回答してください。

**前提知識:**

*   臨床試験においては患者の安全性が最優先され、有害事象の評価は特に重要です。
*   SDTM (Study Data Tabulation Model) は、CDISCによって策定された臨床試験データの標準モデルです。
*   Define.xmlはSDTMデータの構造を記述したメタデータファイルであり、参考情報として使用します。JSONデータ自体の内容、医学的妥当性、プロトコルとの整合性を優先してレビューしてください。
*   SDTMデータは、DM、AE、VS、LBなど、複数のドメイン（データセット）に分かれています。
*   報告されるJSONデータには、データ入力時の間違いが含まれる可能性があります。
*   提供された情報のみに基づいて回答を作成してください。想像やハルシネーションに基づいた回答は作成してはいけません。

**出力形式:**

*   すべての出力はMarkdown形式で作成してください。表形式は使用しないでください。

**その他:**

*   JSONデータまたはDefine.xmlの形式が不正な場合は、その旨をエラーメッセージとして出力してください。
'''




UserInput_Task1 = '''
あなたは臨床試験の専門医です。以下の指示に従い、提供される情報（プロトコル、JSONデータ、Define.xml）を基に、臨床試験データのレビューとクエリ作成（必要な場合）を行ってください。

**1. 試験情報の特定:**

*   **参照情報:** プロトコル、JSONデータ、Define.xml
*   **タスク:**
    *   対象となる臨床試験のフェーズ、疾患領域を特定してください。
    *   有効性評価項目（主要評価項目および副次評価項目）を特定してください。
    *   安全性評価項目を特定してください。
*   **出力形式:** 以下のテンプレートに従ってMarkdown形式で出力してください。

    ```
    *   **試験フェーズ:**
    *   **疾患領域:**
    *   **試験デザイン:** (例：ランダム化、並行群間比較など)
    *   **盲検化:** (例：二重盲検、非盲検など)
    *   **有効性評価項目:**
        *   **主要評価項目:**
        *   **副次評価項目:**
    *   **安全性評価項目:**
    ```

---

**2. 症例サマリーの作成:**

*   **参照情報:** JSONデータ、Define.xml
*   **タスク:**
    *   JSONデータとDefine.xmlを参照し、有害事象、検査値、バイタルサインなどの推移を時系列でまとめた症例サマリーを作成してください。
    *   特に、**異常所見**を中心に簡潔な文章で記載してください。正常範囲内の変動は省略して構いません。
    *   各イベントの日時は、Define.xmlに定義された日付変数などを参考に、正確に特定してください。
*   **出力形式:** 以下のテンプレートに従ってMarkdown形式で出力してください。

    ```
    *   **患者ID:**
        *   YYYY年MM月DD日 (Day XX): [有害事象、検査値、バイタルサインなどのイベントを、異常所見を中心に簡潔な文章で記載]
    ```

---

**3. クエリの作成 (必要な場合のみ):**

*   **参照情報:** JSONデータ、Define.xml、プロトコル
*   **タスク:**
    *   以下のJSONデータのレビュー観点に基づき、JSONデータを改めて点検してください。
    *   医療機関への問い合わせが必要な事項（疑義、不明点、確認事項など）が発生した場合、その内容をまとめたクエリを作成してください。
    *   クエリは、報告されたデータと、Define.xml、プロトコルの記述に基づいて作成してください。提供された情報から逸脱する内容や、想像、ハルシネーションに基づくクエリは作成してはいけません。
    *   クエリは、臨床試験の評価項目に対する影響度を考慮し、重要度の高いものから優先的に作成してください。
    *  **疑義事項がない場合は、クエリを作成する必要はありません。**「疑義事項なし」と回答してください。

*   **JSONデータのレビュー観点 (これらに限定されない):**
    *   **安全性:** 有害事象(AEドメイン)の報告内容は、医学的に妥当であるか？
    *   **医学的妥当性:** 検査値(LBドメイン)の変動、バイタルサイン(VSドメイン)の変動、併用薬(CMドメイン)との相互作用など、時間経過とともに医学的に問題となる点は見られるか？
    *   **有効性:** 特定された主要評価項目および副次評価項目について、その時間的変化は期待される効果と一致しているか？
    *   **その他:** 患者背景(DMドメイン)、既往歴(MHドメイン)、有害事象(AEドメイン)、治療歴(EXドメイン, CMドメイン)などを総合的に考慮し、時間経過を加味して安全性に懸念を生じる事項があれば記載してください。
    *   **プロトコル逸脱 (疑い):** 選択/除外基準、投与量、併用禁止薬、評価スケジュール、有害事象報告などについて、プロトコルからの逸脱の疑いがないか確認してください。（関連ドメイン: DM, MH, EX, CM, LB, VS, AEなど）

*   **出力形式:** 以下のテンプレートに従ってMarkdown形式で出力してください。クエリがない場合は、「クエリなし」と出力してください。

    ```
    *   **患者ID:**
        *   **クエリNo.:**
            *   **臨床試験結果への影響度合い:**（Critical/Major/Minor/None）
            *   **変数名と値:**
            *   **医療機関への問い合わせ文面:**
            *   **判断理由:**
    ```
'''



UserInput_Task2 = '''
あなたはクリニカルデータマネージャーです。以下の指示に従い、提供される情報（JSONデータ、Define.xml、プロトコル）を基に、データ整合性レビューとクエリ作成（必要な場合）を行ってください。

**1. データ整合性レビュー:**

*   **参照情報:** JSONデータ、Define.xml、プロトコル
*   **タスク:**
    *   JSONデータ、Define.xml、プロトコルを参照し、データの不整合が疑われる問題点を検出してください。
    *   **特に、以下の点に焦点を当ててレビューしてください。**
        *   **クロスドメイン整合性:** 異なるSDTMドメイン間で、データに矛盾がないか、ドメイン間の関連性が正しく表現されているか。
            *   **具体的な確認例 (これらに限定されない):**
                *   DM.SEXとAEにおける妊娠関連の有害事象
                *   AEの有害事象発現日や治験薬との関連性と、EXの治験薬の投与期間
                *   LBの検査値異常とAEの関連有害事象
                *   VSのバイタルサイン異常とAEの関連有害事象
                *   CM.CMTRTとAE/MHで報告されている疾患・既往歴との矛盾
        *   **単一ドメイン内の整合性:** Define.xmlの定義に照らして、矛盾なく解釈できるデータになっているか、プロトコルに照らしてデータの関連性が正しく表現されているか。
        *   **異常値:** Define.xmlで定義された範囲外、または医学的にありえない値がないか。
        *   **欠損値:** 欠損値の有無と理由（推測できる場合）。多い場合は原因を推測。
        *   **プロトコル逸脱 (データ品質の観点から):** データ入力/収集で、プロトコルからの逸脱（例：必須項目の未入力、不適切な時期のデータ収集）がないか。
    *   Define.xmlとデータの間に不整合がある場合は、「Define.xmlの修正候補」として報告してください。
*   **出力形式:** 以下のテンプレートに従ってMarkdown形式で出力してください。

    ```
    *   **全体的なデータ品質の評価:**
        *   総合評価: [例：良好、一部問題あり、要修正]
        *   データクリーニング/再調査が必要な項目: [該当項目を列挙]

    *   **問題点:**（問題がある場合）
        *   **問題No.:**
            *   **変数名と値:**
            *   **矛盾の内容:** [具体的な矛盾の内容を記述]
        *   **問題点の原因（推測）:**
        *   **対応策（提案）:**
    ```

**2. クエリの作成 (必要な場合のみ):**

*   **参照情報:** JSONデータ、Define.xml、プロトコル
*   **タスク:**
    *   データ整合性レビューの結果、医療機関への問い合わせが必要な事項（疑義、不明点、確認事項など）が発生した場合、その内容をまとめたクエリを作成してください。
    *   クエリは、報告されたデータと、Define.xml、プロトコルの記述に基づいて作成してください。提供された情報から逸脱する内容や、想像、ハルシネーションに基づくクエリは作成してはいけません。
    *   クエリは、臨床試験の評価項目に対する影響度を考慮し、重要度の高いものから優先的に作成してください。
    *   **疑義事項がない場合は、クエリを作成する必要はありません。**
*   **出力形式:** 以下のテンプレートに従ってMarkdown形式で出力してください。クエリがない場合は、「クエリなし」と出力してください。

    ```
    *   **患者ID:**
        *   **クエリNo.:**
            *   **臨床試験結果への影響度合い:**（Critical/Major/Minor/None）
            *   **変数名と値:**
            *   **医療機関への問い合わせ文面:**
            *   **判断理由:**
    ```
'''


UserInput_Task3 = '''
あなたは、臨床試験の専門医、データマネージャー、CRAの視点を持つ、プロトコル遵守状況の確認者です。以下の指示に従い、提供される情報（JSONデータ、Define.xml、プロトコル）を基に、プロトコル逸脱の検出とクエリ作成（必要な場合）を行ってください。

**1. プロトコル逸脱の検出:**

*   **参照情報:** JSONデータ、Define.xml、プロトコル
*   **タスク:**
    *   JSONデータ、Define.xml、プロトコルを参照し、プロトコルからの逸脱を検出してください。
    *   Define.xmlは参考情報として活用し、データとプロトコルの内容を比較して逸脱を判断してください。
    *   **検出対象とすべき主要なプロトコル逸脱の例 (これらに限定されない):**
        *   **選択/除外基準違反:** (関連SDTMドメイン: DM, MH など)
        *   **投与量違反:** (関連SDTMドメイン: EX)
        *   **併用禁止薬の使用:** (関連SDTMドメイン: CM)
        *   **評価スケジュール違反:** (関連SDTMドメイン: LB, VS, その他)
        *   **有害事象報告違反**: (関連SDTMドメイン: AE)
*   **出力形式:** 以下のテンプレートに従ってMarkdown形式で出力してください。

    ```
    *   **患者ID:**
        *   **逸脱No.:**
            *   **臨床試験結果への影響度合い:**（Critical/Major/Minor）
            *   **変数名と値:**
            *   **逸脱内容:** [具体的な逸脱内容を簡潔に記述。例：被験者XXXは、プロトコルで規定された投与量を超える量の治験薬を投与された]
            *   **プロトコル該当箇所:** [プロトコルの該当するセクション、ページ番号などを記載]
            *   **判断理由:**
    ```

**2. クエリの作成 (必要な場合のみ):**

*   **参照情報:** JSONデータ、Define.xml、プロトコル
*   **タスク:**
    *   プロトコル逸脱を判定するために、医療機関への問い合わせが必要な事項（疑義、不明点、確認事項など）が発生した場合、その内容をまとめたクエリを作成してください。
    *   クエリは、報告されたデータと、Define.xml、プロトコルの記述に基づいて作成してください。提供された情報から逸脱する内容や、想像、ハルシネーションに基づくクエリは作成してはいけません。
    *   クエリは、プロトコル逸脱が臨床試験の評価項目に与える影響度を考慮し、重要度の高いものから優先的に作成してください。
    *   **プロトコル逸脱に関する疑義事項がない場合は、クエリを作成する必要はありません。**
*   **出力形式:** 以下のテンプレートに従ってMarkdown形式で出力してください。クエリがない場合は、「クエリなし」と出力してください。

    ```
    *   **患者ID:**
        *   **クエリNo.:**
            *   **臨床試験結果への影響度合い:**（Critical/Major/Minor/None）
            *   **変数名と値:**
            *   **医療機関への問い合わせ文面:**
            *   **判断理由:**
    ```
'''


UserInput_end1 = '''\n---\n\n**データ:**\n\n*   臨床試験データ（JSON形式、SDTM準拠）:\n\n```json\n'''
UserInput_end2 = '''\n```\n\n*   データ定義ファイル（Define.xml）:\n\n```xml\n''' + define_xml + '''```\n'''

In [13]:
def create_workflow_input(ModelName, SysPrompt, UserInput_Task, datasetjson, UserInput_end1, UserInput_end2):
    return {
        'ModelName': ModelName,
        'SysPrompt': SysPrompt,
        'UserInput': UserInput_Task + UserInput_end1 + datasetjson + UserInput_end2,
        'AttachedFile': {"type": "document", "transfer_method": "local_file", "upload_file_id": "6b06d4f8-d47a-441f-bf67-d8700f76f556"}
    }

## 実行

In [14]:
# ModelNameの設定
#ModelName = 'gemini-2.0-flash'
#ModelName = 'gemini-2.0-flash-exp'
#ModelName = 'gemini-2.0-pro-exp-02-05'
#ModelName = 'gemini-2.0-flash-thinking-exp-01-21'
ModelName = 'gemini-2.0-flash-thinking-exp'


# データ更新症例の抽出
updated_subjects = []
for l in Target_data:
  updated_subjects.append(l[1])

updated_subjects = list(set(updated_subjects))
print(updated_subjects)


['01-703-1335', '01-702-1082', '01-701-1097', '01-701-1023', '01-703-1042', '01-701-1387', '01-703-1076', '01-701-1111', '01-701-1047', '01-701-1180', '01-704-1009', '01-701-1146', '01-701-1153', '01-704-1010', '01-703-1279', '01-703-1086', '01-701-1383', '01-701-1363', '01-703-1096', '01-701-1148', '01-703-1299', '01-701-1118', '01-701-1028', '01-703-1403', '01-701-1034', '01-704-1017', '01-703-1258', '01-701-1181', '01-701-1015', '01-704-1008']
警告：データセット 'CDISCPILOT01.ta' に 'name' が 'USUBJID' の列が見つかりません。スキップします。
警告：データセット 'CDISCPILOT01.te' に 'name' が 'USUBJID' の列が見つかりません。スキップします。
警告：データセット 'CDISCPILOT01.ti' に 'name' が 'USUBJID' の列が見つかりません。スキップします。
警告：データセット 'CDISCPILOT01.ts' に 'name' が 'USUBJID' の列が見つかりません。スキップします。
警告：データセット 'CDISCPILOT01.tv' に 'name' が 'USUBJID' の列が見つかりません。スキップします。
処理完了：'datasetjson' に USUBJID が ['01-702-1082', '01-701-1097'] のデータを出力しました。


In [24]:
import pandas as pd

# リトライ付きでワークフローを実行
results_list = []

for subj in updated_subjects[0:2]:
    datasetjson = filter_data(dataset_list_updated, subj)
    print(f"処理完了：'datasetjson' に USUBJID が {subj} のデータを出力しました。")

    row_data = {'Subject': subj}  # 各行のデータを格納する辞書

    # Task 1 の処理
    workflow_inputs_Task1 = create_workflow_input(ModelName, SysPrompt, UserInput_Task1, UserInput_end1, json.dumps(datasetjson), UserInput_end2)
    result_Task1 = run_workflow_with_retry(api_key, workflow_inputs_Task1, user_id)
    output_Task1 = result_Task1['data']['outputs']['text']
    display(Markdown(output_Task1))
    row_data['Task1'] = output_Task1

    # Task 2 の処理
    workflow_inputs_Task2 = create_workflow_input(ModelName, SysPrompt, UserInput_Task2, UserInput_end1, json.dumps(datasetjson), UserInput_end2)
    result_Task2 = run_workflow_with_retry(api_key, workflow_inputs_Task2, user_id)
    output_Task2 = result_Task2['data']['outputs']['text']
    display(Markdown(output_Task2))
    row_data['Task2'] = output_Task2

    # Task 3 の処理
    workflow_inputs_Task3 = create_workflow_input(ModelName, SysPrompt, UserInput_Task3, UserInput_end1, json.dumps(datasetjson), UserInput_end2)
    result_Task3 = run_workflow_with_retry(api_key, workflow_inputs_Task3, user_id)
    output_Task3 = result_Task3['data']['outputs']['text']
    display(Markdown(output_Task3))
    row_data['Task3'] = output_Task3

    results_list.append(row_data)

# DataFrameを作成
df_results = pd.DataFrame(results_list)

# DataFrameを表示
display(df_results)

警告：データセット 'CDISCPILOT01.ta' に 'name' が 'USUBJID' の列が見つかりません。スキップします。
警告：データセット 'CDISCPILOT01.te' に 'name' が 'USUBJID' の列が見つかりません。スキップします。
警告：データセット 'CDISCPILOT01.ti' に 'name' が 'USUBJID' の列が見つかりません。スキップします。
警告：データセット 'CDISCPILOT01.ts' に 'name' が 'USUBJID' の列が見つかりません。スキップします。
警告：データセット 'CDISCPILOT01.tv' に 'name' が 'USUBJID' の列が見つかりません。スキップします。
処理完了：'datasetjson' に USUBJID が 01-703-1335 のデータを出力しました。
--- 試行回数: 1 ---
Event: workflow_started
------
Event: node_started
Node: 開始
------
Event: node_finished
Node: 開始 (succeeded)
Error: None
Elapsed time: 0.036699
------
Event: node_started
Node: テキスト抽出ツール
------
Event: node_finished
Node: テキスト抽出ツール (succeeded)
Error: None
Elapsed time: 1.214
------
Event: node_started
Node: IF/ELSE
------
Event: node_finished
Node: IF/ELSE (succeeded)
Error: None
Elapsed time: 1.196256
------
Event: parallel_branch_started
------
Event: node_started
Node: (1 T:0.7) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
No

### 1. 試験情報の特定:

*   **試験フェーズ:** Phase II Trial
*   **疾患領域:** アルツハイマー病
*   **試験デザイン:** ランダム化、二重盲検、プラセボ対照、並行群間比較試験
*   **盲検化:** 二重盲検
*   **有効性評価項目:**
    *   **主要評価項目:**
        *   ADAS-Cog (11項目版) スコアのベースラインからの変化量
        *   CIBIC+ スコア
    *   **副次評価項目:**
        *   NPI-X (Revised Neuropsychiatric Inventory) スコア
        *   DAD (Disability Assessment for Dementia) スコア
        *   ADAS-Cog (14項目版) スコア
        *   Apo E 遺伝子型別の治療反応
*   **安全性評価項目:**
    *   有害事象 (AE)
    *   臨床検査値 (LB)
    *   バイタルサイン (VS)
    *   心電図 (ECG、Ambulatory ECG)

### 2. 症例サマリーの作成:

*   **患者ID:** 01-703-1335
    *   2014年02月21日 (Day -24): 血中クレアチンキナーゼ (CK) が基準値上限超え (209 U/L)。
    *   2014年03月15日 (Day -2): 軽度の房室ブロック (Atrioventricular block second degree) および多発性硬化症再発 (Multiple sclerosis relapse) の有害事象が発現 (いずれも軽度)。
    *   2014年03月30日 (Day 14): 房室ブロック (Atrioventricular block second degree) の有害事象が回復。
    *   2014年03月31日 (Day 15): 試験薬をXanomeline 81mgへ増量。血中クレアチンキナーゼ (CK) が再び基準値上限超え (219 U/L)。
    *   2014年05月01日 (Day 46): 多発性硬化症再発 (Multiple sclerosis relapse) および房室ブロック (Atrioventricular block second degree) の有害事象が未回復のまま持続。
    *   2014年05月24日 (Day 69): プロトコル違反 (PROTOCOL VIOLATION) により試験中止。最終検査 (FINAL LAB VISIT) を実施。最終検査時の検査値では、総コレステロール (CHOL) が基準値上限超え (262 mg/dL)、尿比重 (SPGRAV) が基準値下限を下回る (1.016) 以外は、概ね基準値内。

### 3. クエリの作成:

*   **患者ID:** 01-703-1335
    *   **クエリNo.:** 1
        *   **臨床試験結果への影響度合い:** Critical
        *   **変数名と値:** DS.DSTERM = PROTOCOL VIOLATION, DS.DSDECOD = PROTOCOL VIOLATION
        *   **医療機関への問い合わせ文面:**
            プロトコル違反による試験中止の理由と詳細についてご教示ください。特に、違反内容、発生日、状況、治験薬の投与量、投与期間、選択基準・除外基準との関連性、有害事象との関連性など、試験結果に与える可能性のある影響について詳しく教えてください。
        *   **判断理由:**
            プロトコル違反による試験中止は、試験データの解釈、特に安全性評価と有効性評価に重大な影響を与える可能性があるため、詳細な理由と状況を把握し、評価に反映させる必要

--- 試行回数: 1 ---
Event: workflow_started
------
Event: node_started
Node: 開始
------
Event: node_finished
Node: 開始 (succeeded)
Error: None
Elapsed time: 0.035591
------
Event: node_started
Node: テキスト抽出ツール
------
Event: node_finished
Node: テキスト抽出ツール (succeeded)
Error: None
Elapsed time: 0.236014
------
Event: node_started
Node: IF/ELSE
------
Event: node_finished
Node: IF/ELSE (succeeded)
Error: None
Elapsed time: 0.119314
------
Event: parallel_branch_started
------
Event: node_started
Node: (1 T:0.7) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (5 T:0.3) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (4 T:0.5) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (3 T:0.7) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (2 T:0.7) gemini-2.0-flash-thinking-exp 
------
Event: node_f

はい、承知いたしました。5人のアシスタントの回答を統合し、指定のフォーマットで出力します。

*   **全体的なデータ品質の評価:**
    *   総合評価: 一部問題あり
    *   データクリーニング/再調査が必要な項目: AE.AESTDTC, AE.AEENDTC, LB.LBORRES(CK), DS.DSTERM, MH.MHSTDTC, EX.EXSTDTC, EX.EXENDTC, DS.DSDTC, LB.LBDTC, QS.QSDTC, SV.SVSTDTC, SV.SVENDTC, CM.CMDTC, MH.MHDTC, SC.SCDTC, VS.VSDTC, AE.AEDTC, DS.DSDTCの日付データ

*   **問題点:**
    *   **問題No.1:**
        *   **変数名と値:**
            *   AE.USUBJID = 01-703-1335, AE.AESEQ = 1, AE.AESTDTC = 2014-04-01, EX.EXENDTC = 2014-03-31
        *   **矛盾の内容:** 有害事象（AESEQ=1）の開始日（2014-04-01）が、治験薬投与期間の終了日（2014-03-31）より後になっている。治験薬との因果関係を再確認する必要がある。
        *   **問題点の原因（推測）:** データ入力時の誤り、または治験薬と関連性のない有害事象の可能性。
        *   **対応策（提案）:** 医療機関にAE開始日と治験薬投与期間の矛盾について確認し、データ入力誤りがないか確認する。データ入力誤りでない場合、医学的レビューを実施する。

    *   **問題No.2:**
        *   **変数名と値:**
            *   AE.USUBJID = 01-703-1335, AE.AESEQ = 2, AE.AESTDTC = 2014-03-15, EX.EXSTDTC = 2014-03-17
        *   **矛盾の内容:** 有害事象（AESEQ=2）の開始日（2014-03-15）が、治験薬投与開始日（2014-03-17）より前になっている。治験薬との因果関係を再確認する必要がある。また、AEレコード1との日付の整合性も要確認。
        *   **問題点の原因（推測）:** データ入力時の誤り、または日付データの解釈の誤り。
        *   **対応策（提案）:** 症例記録（CRF）を確認し、AE開始日、治験薬投与期間、VISIT情報を再確認する。必要に応じて医療機関に問い合わせ、データ修正の要否を確認する。

    *   **問題No.3:**
        *   **変数名と値:**
            *   LB.LBTESTCD = CK, LB.LBORRES = 209, 219 (Visit 1.1, Visit 4)
        *   **矛盾の内容:** LBドメインのCK検査値が、Visit 1.1とVisit 4で基準値上限を超過 (HIGH) している。臨床的な意義と関連する有害事象の有無を確認する必要がある。
        *   **問題点の原因（推測）:** 検査値の一時的な変動、または薬剤性ミオパチー等の可能性。
        *   **対応策（提案）:** CK値が基準値上限を超過していることの臨床的な意義について、治験責任医師に確認する。関連するAEドメインのデータも確認する。

    *   **問題No.4:**
        *   **変数名と値:** DS.DSTERM = PROTOCOL VIOLATION, DS.DSDECOD = PROTOCOL VIOLATION
        *   **矛盾の内容:** DSドメインにプロトコル逸脱が報告されているが、理由や詳細が不明。データ品質に影響を与える可能性があるため、内容を把握する必要がある。
        *   **問題点の原因（推測）:** データ入力時の情報不足、またはプロトコル逸脱の理由がデータに記録されていない可能性。
        *   **対応策（提案）:** PROTOCOL VIOLATION の詳細な理由と臨床試験への影響について、医療機関に問い合わせる。

    *   **問題No.5:**
        *   **変数名と値:** MH.MHSTDTC = (欠損)
        *   **矛盾の内容:** MHドメインのMHSTDTC (Medical History Event Start Date/Time) が欠損。重要な既往歴情報であるため、可能な限り正確なデータを収集する必要がある。
        *   **問題点の原因（推測）:** データ入力時の必須項目漏れ、または情報収集時の未取得。
        *   **対応策（提案）:** MHSTDTC が欠損している理由を確認し、可能であれば医療機関に正確な開始日を問い合わせる。

    *   **問題No.6:**
        *   **変数名と値:** EXドメインのEXSTDTC, EXENDTC, TEドメインのTEDUR
        *   **矛盾の内容:** EXドメインの治験薬投与期間がTEドメインのTEDUR（2週間）と一致しない。プロトコル逸脱の可能性。
        *   **問題点の原因（推測）:** プロトコルからの逸脱、またはデータ入力時の誤り。
        *   **対応策（提案）:** プロトコルを確認し、投与期間の逸脱が意図的なものかどうかを確認する。

    *   **問題No.7:**
        *   **変数名と値:** DSドメインのDSDTC (Row 1)
        *   **矛盾の内容:** DSドメインのデータ収集日時（DSDTC）において、一部レコードで時間情報が欠損している。データ型の定義と実際のデータ形式が一致しているか確認する必要がある。
        *   **問題点の原因（推測）:** データ入力時の誤り、またはデータ収集システムの不具合。
        *   **対応策（提案）:** データ入力者に確認し、時間情報の欠損理由を特定する。Define.xmlとJSONデータ定義のdataTypeの整合性を確認する。

    *   **問題No.8:**
        *   **変数名と値:** 各ドメインの日付データ (DTC変数)
        *   **矛盾の内容:** 複数ドメインで日付データ（DTC変数）の形式がdate型とdatetime型で混在している。データ形式の統一とDefine.xmlの定義との整合性を確認する必要がある。
        *   **問題点の原因（推測）:** Define.xmlの定義誤り、JSONデータ生成ロジックの誤り、またはデータ入力時の形式のばらつき。
        *   **対応策（提案）:** Define.xmlの定義とJSONデータのdataType定義を比較し、矛盾を解消する。データ入力者への日付形式統一の徹底、Define.xmlの修正（datetime型への統一）を検討する。

*   **Define.xmlの修正候補:**
    *   日付関連のItemDef (CMDTC, DMDTC, MHDTC, SCDTC, VSDTC, AEDTC, DSDTC, EXSTDTC, EXENDTC, SESTDTC, SEENDTC, SVSTDTC, SVENDTC, QSDTC, LBDTC) のdataTypeをdatetime型に修正

*   **患者ID:** 01-703-1335
    *   **クエリNo.1:**
        *   **臨床試験結果への影響度合い:** Major
        *   **変数名と値:** AE.AESTDTC = 2014-04-01, EX.EXENDTC = 2014-03-31 (AESEQ=1)
        *   **医療機関への問い合わせ文面:** 患者ID 01-703-1335 の有害事象報告（AESEQ=1）について、開始日（2014-04-01）が治験薬投与終了日（2014-03-31）より後になっています。AE開始日、治験薬投与期間に誤りがないか、原資料をご確認ください。
        *   **判断理由:** 有害事象と治験薬の因果関係評価に重大な影響を及ぼす可能性があり、データの信頼性を損なうため。

    *   **クエリNo.2:**
        *   **臨床試験結果への影響度合い:** Major
        *   **変数名と値:** AE.AESTDTC = 2014-03-15, EX.EXSTDTC = 2014-03-17 (AESEQ=2)
        *   **医療機関への問い合わせ文面:** 患者ID 01-703-1335 の有害事象報告（AESEQ=2）について、開始日（2014-03-15）が治験薬投与開始日（2014-03-17）より前になっています。AE開始日、治験薬投与期間に誤りがないか、原資料をご確認ください。
        *   **判断理由:** 有害事象と治験薬の因果関係評価に重大な影響を及ぼす可能性があり、データの信頼性を損なうため。

    *   **クエリNo.3:**
        *   **臨床試験結果への影響度合い:** Major
        *   **変数名と値:** LB.LBTESTCD = CK, LB.LBORRES = 209, 219
        *   **医療機関への問い合わせ文面:** 患者ID 01-703-1335 の臨床検査値データで、CK (Creatine Kinase) がVisit 1.1とVisit 4で基準値上限を超過しています。臨床的な意義と、関連する有害事象の有無についてご教示ください。
        *   **判断理由:** CK値上昇は患者安全性に関わる可能性があり、臨床的評価が必要なため。

    *   **クエリNo.4:**
        *   **臨床試験結果への影響度合い:** Minor
        *   **変数名と値:** DS.DSTERM = PROTOCOL VIOLATION
        *   **医療機関への問い合わせ文面:** 患者ID 01-703-1335 のDispositionデータにあるPROTOCOL VIOLATION (プロトコル逸脱) の詳細な理由と内容についてご教示ください。
        *   **判断理由:** プロトコル逸脱の内容を把握し、データ品質への影響を評価するため。

    *   **クエリNo.5:**
        *   **臨床試験結果への影響度合い:** Minor
        *   **変数名と値:** MH.MHSTDTC (欠損)
        *   **医療機関への問い合わせ文面:** 患者ID 01-703-1335 のMedical Historyデータ (MHSEQ=1) で開始日 (MHSTDTC) が欠損しています。可能であれば開始日をご教示ください。
        *   **判断理由:** 既往歴の開始日は重要な情報であり、データ補完のため可能な範囲で情報を収集するため。

以上、データ整合性レビューとクエリ作成の結果です。

--- 試行回数: 1 ---
Event: workflow_started
------
Event: node_started
Node: 開始
------
Event: node_finished
Node: 開始 (succeeded)
Error: None
Elapsed time: 0.037423
------
Event: node_started
Node: テキスト抽出ツール
------
Event: node_finished
Node: テキスト抽出ツール (succeeded)
Error: None
Elapsed time: 0.183344
------
Event: node_started
Node: IF/ELSE
------
Event: node_finished
Node: IF/ELSE (succeeded)
Error: None
Elapsed time: 0.178482
------
Event: parallel_branch_started
------
Event: node_started
Node: (1 T:0.7) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (5 T:0.3) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (4 T:0.5) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (3 T:0.7) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (2 T:0.7) gemini-2.0-flash-thinking-exp 
------
Event: node_f

...

--- 試行回数: 1 ---
Event: workflow_started
------
Event: node_started
Node: 開始
------
Event: node_finished
Node: 開始 (succeeded)
Error: None
Elapsed time: 0.03886
------
Event: node_started
Node: テキスト抽出ツール
------
Event: node_finished
Node: テキスト抽出ツール (succeeded)
Error: None
Elapsed time: 0.202133
------
Event: node_started
Node: IF/ELSE
------
Event: node_finished
Node: IF/ELSE (succeeded)
Error: None
Elapsed time: 0.239135
------
Event: parallel_branch_started
------
Event: node_started
Node: (1 T:0.7) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (5 T:0.3) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (4 T:0.5) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (3 T:0.7) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (2 T:0.7) gemini-2.0-flash-thinking-exp 
------
Event: node_fi

はい、承知いたしました。5人のアシスタントの回答を統合し、所見ごとに指定のフォーマットに従って出力します。

*   **全体的なデータ品質の評価:**
    *   総合評価: 一部問題あり
    *   データクリーニング/再調査が必要な項目: CM.CMENDTC, DS.DSTERM, LBドメインの異常値とAEの関連性, LB.VISITDY (UNSCHEDULED 1.1), VSドメインの欠損値, VS.VSORRES ("0" が先頭に付与されているデータ)

*   **問題点:**
    *   **問題No.1:**
        *   **変数名と値:** VS.VSTESTCD="DIABP", VS.VSTESTCD="PULSE", VS.VSTESTCD="SYSBP", VS.VSPOS="STANDING", VISIT="SCREENING 2" における VSORRES の欠損 (レコード5, 66, 36)
        *   **矛盾の内容:** VISIT="SCREENING 2" における、立位でのDiastolic Blood Pressure、Pulse Rate、Systolic Blood Pressureの測定値が欠損しており、VSSTAT が "NOT DONE" と記録されているレコードが存在します。プロトコルでは立位でのバイタルサイン測定が必須である可能性があり、データ入力時のエラーまたは測定自体の未実施が疑われます。
        *   **問題点の原因（推測）:** データ入力エラー、測定忘れ、患者の状態により測定不能だった、またはUNSCHEDULED VISITのためVSITDYが定義されていない可能性などが考えられます。
        *   **対応策（提案）:** 医療機関に記録を確認し、データ欠損の理由、測定の実施有無、データ入力エラーの可能性について確認してください。可能であればデータ修正を依頼し、測定未実施の場合は理由を確認してください。プロトコルで必須の検査項目であるか確認し、必須であれば再測定を検討するか、欠損値として扱う理由を明確にしてください。データ収集手順、データ入力手順を見直し、同様の欠損が再発しないように対策を講じることも推奨されます。

    *   **問題No.2:**
        *   **変数名と値:** CM.CMENDTC (レコード1, 2)
        *   **矛盾の内容:** CM.CMENDTC（併用薬終了日）が CMDTC（データ収集日）より過去の日付になっているレコード（1行目、2行目）が存在します。CMDTC はデータ収集日であり、CMENDTC は併用薬終了日であるため、時系列に矛盾が生じています。
        *   **問題点の原因（推測）:** データ入力時の誤り、データ収集日の誤り、または意図的な過去の日付入力の可能性があります。
        *   **対応策（提案）:** CM.CMENDTC と CM.CMDTC の日付の整合性を医療機関に確認し、データ入力ミスであれば修正を依頼してください。データ収集日、併用薬終了日について記録を確認し、必要に応じて修正を行ってください。

    *   **問題No.3:**
        *   **変数名と値:** LB.LBTESTCD="ALT", LB.LBTESTCD="AST", LB.LBTESTCD="BUN", LB.LBTESTCD="CK", LB.LBTESTCD="HCT", LB.LBTESTCD="WBC", LB.LBTESTCD="PROT", LB.LBTESTCD="VITB12" で LBNRIND="HIGH" のレコード (レコード16, 17, 24, 25, 26, 48, 54, 228, 37, 73, 168, 198, 233)
        *   **矛盾の内容:** 複数の臨床検査項目（ALT, AST, BUN, CK, HCT, WBC, PROT, VITB12）で基準値上限を超える異常値 (LBNRIND="HIGH") が確認されました。AEドメインには関連する有害事象が報告されているものの、LBドメインの異常値とAEドメインの関連性が明確ではありません。また、これらの検査値異常が治験薬の影響、原疾患によるもの、または臨床的に注意が必要な医学的妥当性の観点から精査が必要です。
        *   **問題点の原因（推測）:** 治験薬の影響、原疾患の病態変化、検査値の一時的な変動、データ入力エラー、またはAE報告の不備などが考えられます。
        *   **対応策（提案）:** 医療機関にLBNRIND="HIGH" となっている検査値異常について、以下の点を確認するクエリを発行し、回答を

--- 試行回数: 1 ---
Event: workflow_started
------
Event: node_started
Node: 開始
------
Event: node_finished
Node: 開始 (succeeded)
Error: None
Elapsed time: 0.044331
------
Event: node_started
Node: テキスト抽出ツール
------
Event: node_finished
Node: テキスト抽出ツール (succeeded)
Error: None
Elapsed time: 0.27551
------
Event: node_started
Node: IF/ELSE
------
Event: node_finished
Node: IF/ELSE (succeeded)
Error: None
Elapsed time: 0.155689
------
Event: parallel_branch_started
------
Event: node_started
Node: (1 T:0.7) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (5 T:0.3) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (4 T:0.5) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (3 T:0.7) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (2 T:0.7) gemini-2.0-flash-thinking-exp 
------
Event: node_fi

はい、承知いたしました。5人のアシスタントの回答を統合し、指定されたフォーマットで出力します。

## プロトコル逸脱の検出

*   **患者ID:** 01-702-1082
    *   **逸脱No.:** 1
        *   **臨床試験結果への影響度合い:** Major
        *   **変数名と値:** MH.MHTERM = CARDIAC PACEMAKER INSERTION, MH.MHTERM = AORTIC STENOSIS
        *   **逸脱内容:** Medical History に CARDIAC PACEMAKER INSERTION (心臓ペースメーカー挿入) と AORTIC STENOSIS (大動脈弁狭窄症) の記載があり、プロトコルの除外基準 EXCL17 「A history within the last 5 years of a serious cardiovascular disorder」に抵触する可能性があります。
        *   **プロトコル該当箇所:** 3.4.2.2. Exclusion Criteria [17]
        *   **判断理由:** 心血管系の既往歴は、治験薬の安全性に影響を与える可能性があるため、重大なプロトコル逸脱の疑いとして検出しました。特にAORTIC STENOSIS (大動脈弁狭窄症) は、重篤な心血管疾患に該当する可能性があり、選択基準/除外基準の遵守状況を確認する必要があります。

    *   **逸脱No.:** 2
        *   **臨床試験結果への影響度合い:** Minor
        *   **変数名と値:** CM.CMTRT = HYDROCORTISONE, TOPICAL
        *   **逸脱内容:** WEEK 8 および WEEK 12 に HYDROCORTISONE, TOPICAL (局所用ハイドロコルチゾン) の併用が記録されています。プロトコルに併用禁止薬に関する明確な記述はありませんが、HYDROCORTISONE, TOPICAL の使用がプロトコルで許容されているか不明です。
        *   **プロトコル該当箇所:** 3.8. Concomitant Therapy
        *   **判断理由:** 併用薬 HYDROCORTISONE, TOPICAL の臨床試験への影響が不明なため、プロトコル逸脱の疑いとして検出しました。Define.xml の CMCLAS に "UNCODED" と記載されており、薬剤分類が特定できない点も懸念されます。

    *   **逸脱No.:** 3
        *   **臨床試験結果への影響度合い:** Minor
        *   **変数名と値:** LB.VISIT = UNSCHEDULED 1.1, LB.LBTESTCD=ALT, LB.LBORRES=37 U/L, LB.LBTESTCD=AST, LB.LBORRES=44 U/L, LB.LBTESTCD=BUN, LB.LBORRES=27 mg/dL
        *   **逸脱内容:** 計画外の VISIT (UNSCHEDULED 1.1) において、ALT, AST, BUN の検査値が基準範囲上限を超過しています。プロトコル除外基準 EXCL27 に抵触する可能性があり、また評価スケジュールからの逸脱も懸念されます。
        *   **プロトコル該当箇所:** 3.4.2.2. Exclusion Criteria [27b], 3.9.3.3 臨床検査, Protocol Attachment LZZT.1. Schedule of Events for Protocol H2Q-MC-LZZT(c)
        *   **判断理由:** 肝機能・腎機能検査値の基準値超過は安全性上の懸念があり、プロトコル逸脱および選択基準違反の疑いとして検出しました。また、計画外の VISIT での検査実施は評価スケジュールからの逸脱の可能性があります。

    *   **逸脱No.:** 4
        *   **臨床試験結果への影響度合い:** Minor
        *   **変数名と値:** LB.LBTESTCD=CK, LBORRES=320, 177 U/L (WEEK 2, WEEK 8)
        *   **逸脱内容:** WEEK 2 および WEEK 8 において、CK値が基準範囲上限を超過しています。プロトコル除外基準 EXCL27 に抵触する可能性があります。
        *   **プロトコル該当箇所:** 3.4.2.2. Exclusion Criteria [27b], 3.9.3.3 臨床検査
        *   **判断理由:**  CK値の基準値超過は安全性上の懸念があり、プロトコル逸脱および選択基準違反の疑いとして検出しました。ただし、CK値はその後正常範囲内まで低下しており、臨床的な意義は Minor と考えられます。

    *   **逸脱No.:** 5
        *   **臨床試験結果への影響度合い:** Minor
        *   **変数名と値:** LB.LBTESTCD=HCT, LBORRES=49.0 % (SCREENING 1), LB.LBTESTCD=WBC, LBORRES=14.77 THOU/uL (SCREENING 1)
        *   **逸脱内容:** SCREENING 1 において、HCT および WBC 値が基準範囲上限を超過しています。プロトコル除外基準 EXCL27 に抵触する可能性があります。
        *   **プロトコル該当箇所:** 3.4.2.2. Exclusion Criteria [27b], 3.9.3.3 臨床検査
        *   **判断理由:**  HCT および WBC 値の基準値超過は安全性上の懸念があり、プロトコル逸脱および選択基準違反の疑いとして検出しました。ただし、これらの値はベースライン以降で正常範囲内まで低下しており、臨床的な意義は Minor と考えられます。

    *   **逸脱No.:** 6
        *   **臨床試験結果への影響度合い:** Minor
        *   **変数名と値:** VS.VSTESTCD=DIABP, VS.VISIT=SCREENING 2, VS.VSPOS=STANDING, VS.VSSTSTAT=NOT DONE, VS.VSTESTCD=SYSBP, VS.VISIT=SCREENING 2, VS.VSPOS=STANDING, VS.VSSTSTAT=NOT DONE
        *   **逸脱内容:** SCREENING 2 (VISITNUM=2) において、立位 (STANDING) での拡張期血圧 (DIABP) および収縮期血圧 (SYSBP) の測定が実施されていません。
        *   **プロトコル該当箇所:** 3.9.3.4.1 Vital Sign Determination, Protocol Attachment LZZT.1. Schedule of Events for Protocol H2Q-MC-LZZT(c)
        *   **判断理由:** プロトコルで規定された評価項目（立位での血圧測定）が実施されていない可能性があります。安全性評価項目の一部の欠落として、プロトコル逸脱の疑いがあると判断しました。

## クエリの作成

*   **患者ID:** 01-702-1082
    *   **クエリNo.:** 1
        *   **臨床試験結果への影響度合い:** Major
        *   **変数名と値:** MH.MHTERM = CARDIAC PACEMAKER INSERTION, MH.MHTERM = AORTIC STENOSIS
        *   **医療機関への問い合わせ文面:** 被験者 01-702-1082 の Medical History に記載された CARDIAC PACEMAKER INSERTION (心臓ペースメーカー挿入) と AORTIC STENOSIS (大動脈弁狭窄症) について、以下の情報をご提供ください。
            1.  それぞれの具体的な診断名、発症時期、重症度、現在の病状
            2.  AORTIC STENOSIS は、プロトコル除外基準 EXCL17 「A history within the last 5 years of a serious cardiovascular disorder」に該当するか否かの治験責任医師の見解
            3.  心臓ペースメーカー挿入の理由と、治験への参加に影響がないと判断された医学的根拠
        *   **判断理由:** 重大な心血管系既往歴がプロトコル除外基準に抵触する可能性があるため、詳細な情報を収集し、治験継続の妥当性を医学的に評価する必要がある。

    *   **クエリNo.:** 2
        *   **臨床試験結果への影響度合い:** Minor
        *   **変数名と値:** CM.CMTRT = HYDROCORTISONE, TOPICAL, CM.CMCLAS = UNCODED
        *   **医療機関への問い合わせ文面:** 被験者 01-702-1082 の併用薬 HYDROCORTISONE, TOPICAL (局所用ハイドロコルチゾン) について、MedDRA コード、ATC コード、薬剤分類 (CMCLAS) および使用目的 (CMINDC) をご教示ください。また、本剤がプロトコルで許容される併用薬に該当するか、あるいは臨床試験に影響を与える併用禁止薬に該当するかについて、治験責任医師の見解をご教示ください。併用が許可されている場合は、その理由と医学的妥当性についてもご説明ください。
        *   **判断理由:** 併用薬 HYDROCORTISONE, TOPICAL の詳細情報を収集し、プロトコル遵守状況と臨床試験への影響を評価するため。

    *   **クエリNo.:** 3
        *   **臨床試験結果への影響度合い:** Major
        *   **変数名と値:** LB.LBTESTCD=ALT, LB.LBORRES=37 U/L (UNSCHEDULED 1.1), LB.LBTESTCD=AST, LB.LBORRES=44 U/L (UNSCHEDULED 1.1), LB.LBTESTCD=BUN, LB.LBORRES=27 mg/dL (UNSCHEDULED 1.1)
        *   **医療機関への問い合わせ文面:** 治験参加者 01-702-1082 の UNSCHEDULED VISIT 1.1 (2013-07-24) における肝機能検査 (ALT, AST) および腎機能検査 (BUN) 値の基準値上限超過について、以下の点をご回答ください。
            1.  UNSCHEDULED VISIT 1.1 を実施した理由と、実施された臨床検査項目の医学的妥当性
            2.  基準値超過が Lilly Reference Range III の基準に照らして除外基準 [27b] に該当するか否かの治験責任医師の見解
            3.  除外基準に該当する場合、治験継続の判断に至った理由と医学的妥当性
            4.  肝機能・腎機能検査値上昇の原因、および治験薬投与との関連性についての治験責任医師の見解
        *   **判断理由:** 基準値外の臨床検査値が複数項目で確認されたため、プロトコル除外基準、評価スケジュール遵守状況、および臨床試験への影響を総合的に評価するため、詳細な情報を医療機関に問い合わせる必要がある。

    *   **クエリNo.:** 4
        *   **臨床試験結果への影響度合い:** Minor
        *   **変数名と値:** VS.VSTESTCD=DIABP, VS.VISIT=SCREENING 2, VS.VSPOS=STANDING, VS.VSSTSTAT=NOT DONE, VS.VSTESTCD=SYSBP, VS.VISIT=SCREENING 2, VS.VSPOS=STANDING, VS.VSSTSTAT=NOT DONE
        *   **医療機関への問い合わせ文面:** 治験参加者 01-702-1082 の SCREENING 2 (VISITNUM=2) における立位での拡張期血圧 (DIABP) および収縮期血圧 (SYSBP) が未実施となっています。測定未実施の理由、およびデータ入力エラーの可能性についてご回答ください。測定が未実施の場合、医学的な理由と今後の対応についてご説明ください。
        *   **判断理由:** プロトコルで規定されたバイタルサイン測定が未実施である理由を確認し、データの欠測、評価スケジュールからの逸脱、および今後の対応について確認するため。

---
クエリなし

,Subject,Task1,Task2,Task3
0,01-703-1335,### 1. 試験情報の特定:\n\n* **試験フェーズ:** Phase II Tr...,はい、承知いたしました。5人のアシスタントの回答を統合し、指定のフォーマットで出力します。\...,はい、承知いたしました。5人のアシスタントの回答を統合し、以下のMarkdown形式で出力し...
1,01-702-1082,```markdown\n## 臨床試験データレビュー結果（統合）\n\n### 1. 試験...,はい、承知いたしました。5人のアシスタントの回答を統合し、所見ごとに指定のフォーマットに従っ...,はい、承知いたしました。5人のアシスタントの回答を統合し、指定されたフォーマットで出力します...


In [25]:
import pandas as pd

def dataframe_to_text(df: pd.DataFrame) -> str:
    """
    DataFrameを指定されたテキスト形式に変換します。

    Args:
        df: 変換するDataFrame。カラム名は 'Subject', 'Task1', 'Task2', 'Task3' である必要があります。

    Returns:
        変換後のテキストデータ。
    """
    text_data = ""
    for index, row in df.iterrows():
        subject = row['Subject']
        task1 = row['Task1']
        task2 = row['Task2']
        task3 = row['Task3']

        text_data += f"# {subject}\n"
        text_data += f"## Task1\n"
        text_data += f"{task1}\n"
        text_data += f"## Task2\n"
        text_data += f"{task2}\n"
        text_data += f"## Task3\n"
        text_data += f"{task3}\n\n"

    return text_data

# サンプルDataFrameの作成 (提供されたデータを元に)
data = {'Subject': ['001-703-1335', '01-702-1082', '01-701-1097'],
        'Task1': ['はい、承知いたしました。5人のアシスタントの回答を統合し、指定されたフォーマットで出力します...',
                  'はい、承知いたしました。5人のアシスタントの回答を統合し、指定されたMarkdown形式で出...',
                  '承知いたしました。5人のアシスタントの回答を統合し、指定された形式で出力します。\n\n--...'],
        'Task2': ['はい、承知いたしました。5人のアシスタントの回答を統合し、指定のフォーマットで出力します。\\...',
                  '```markdown\n* **患者ID:** 01-702-1082\n * ...',
                  '* **全体的なデータ品質の評価:**\n * 総合評価: 一部問題あり\n ...'],
        'Task3': ['### プロトコル逸脱の検出\n\n* **患者ID:** 01-703-1335\n ...',
                  '',  # Task3のデータがない場合
                  '### プロトコル逸脱の検出:\n\n* **患者ID:** 01-701-1097\n...']}
df = pd.DataFrame(data)

# DataFrameをテキストデータに変換
output_text = dataframe_to_text(df)

# 結果の出力
print(output_text)

# 001-703-1335
## Task1
はい、承知いたしました。5人のアシスタントの回答を統合し、指定されたフォーマットで出力します...
## Task2
はい、承知いたしました。5人のアシスタントの回答を統合し、指定のフォーマットで出力します。\...
## Task3
### プロトコル逸脱の検出

* **患者ID:** 01-703-1335
 ...

# 01-702-1082
## Task1
はい、承知いたしました。5人のアシスタントの回答を統合し、指定されたMarkdown形式で出...
## Task2
```markdown
* **患者ID:** 01-702-1082
 * ...
## Task3


# 01-701-1097
## Task1
承知いたしました。5人のアシスタントの回答を統合し、指定された形式で出力します。

--...
## Task2
* **全体的なデータ品質の評価:**
 * 総合評価: 一部問題あり
 ...
## Task3
### プロトコル逸脱の検出:

* **患者ID:** 01-701-1097
...


